In [1]:
import numpy as np #importamos numpy para un mejor manejo de matrizes, filas y valores
import matplotlib.pyplot as plt #esta libreria la usaremos para graficar
from pyspark.sql import SparkSession #PySpark sera nuestra opcion para el manejo de grandes volumenes de datos
import os
import pandas

# Crea una sesión de Spark
spark = SparkSession.builder \
    .appName("MiAplicacionSpark") \
    .getOrCreate()

# Importamos las funciones de sql de pyspark para manejar mejor las columnas
from pyspark.sql.functions import col, sum, when, size, length, to_date, from_unixtime, year, count

from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, BooleanType

import importlib
import utils.validation

#Reload the functions module
importlib.reload(utils.validation)

In [2]:
from automatizacion_reviews import etl_reviews
from automatizacion_metadata import etl_metadata

In [3]:
df = etl_reviews('../datasets/Digital_Music.json')

In [4]:
df2 = etl_reviews('../datasets/All_Beauty.json')

In [5]:
df3 = etl_reviews('../datasets/Appliances.json')

In [6]:
df4 = etl_reviews('../datasets/Musical_Instruments.json')

In [16]:
df.show(5)
df2.show(5)
df3.show(5)
df4.show(5)

+----------+-------+--------------------+--------------+--------------+--------+-------------------+-----------+
|      asin|overall|          reviewText|    reviewerID|  reviewerName|verified|        review_date|review_year|
+----------+-------+--------------------+--------------+--------------+--------+-------------------+-----------+
|B000V6AEI6|    5.0|The composistions...| AAVK0CAPD9XH5|          mole|   false|2003-12-31 21:00:00|       2003|
|B000VRWV4U|    5.0|I needed some sca...|A3O39R049OZZUZ|         Wonka|    true|2012-12-31 21:00:00|       2012|
|B000TE3FBS|    1.0|never received it...| A19CD6KQG33SD|         Dixie|    true|2013-12-31 21:00:00|       2013|
|B0048W3SQ4|    5.0|WHAT ELSE CAN YOU...|A1WHQ7EFI717UH|JOHNWAYNEFAN57|    true|2013-01-09 21:00:00|       2013|
|B0011W0BKA|    5.0|use this for our ...|A2JTT6E8TMNUIG|Family of Five|    true|2013-01-10 21:00:00|       2013|
+----------+-------+--------------------+--------------+--------------+--------+----------------

In [17]:
df_meta = etl_metadata('../datasets/meta_Digital_Music.json')
df_meta2 = etl_metadata('../datasets/meta_All_Beauty.json')
df_meta3 = etl_metadata('../datasets/meta_Appliances.json')
df_meta4 = etl_metadata('../datasets/meta_Musical_Instruments.json')

In [9]:
df_meta.show(3)
df_meta3.show(3)
df_meta2.show(3)

+--------+---------+----------+-------------------+-----------+--------------------+-------------+-----+-------+--------------------+
|also_buy|also_view|      asin|              brand|description|             details|     main_cat|price|   rank|               title|
+--------+---------+----------+-------------------+-----------+--------------------+-------------+-----+-------+--------------------+
|     N/D|      N/D|B000003SDF|    Richard H. Kirk|        N/D|{null, null, B000...|Digital Music| 9.99|1255550|Agents With False...|
|     N/D|      N/D|B000006XV0|           Creation|        N/D|{null, null, B000...|Digital Music| 12.5|1761232|       Lay the Ghost|
|     N/D|      N/D|B00000AQFD|Wildecker Herzbuben|        N/D|{null, null, B000...|Digital Music| 1.72|1157548|Ist das nicht him...|
+--------+---------+----------+-------------------+-----------+--------------------+-------------+-----+-------+--------------------+
only showing top 3 rows

+--------+---------+----------+------

In [18]:
df_meta4.show()

+--------------------+---------+----------+--------------------+--------------------+--------------------+-------------+------+--------------------+--------------------+
|            also_buy|also_view|      asin|               brand|         description|             details|     main_cat| price|                rank|               title|
+--------------------+---------+----------+--------------------+--------------------+--------------------+-------------+------+--------------------+--------------------+
|                 N/D|      N/D|B0002F5DH8|                    |                 N/D|{null, null, null...|Digital Music|  0.91|       5041631001233|LMI 3-Piece Alto ...|
|                 N/D|      N/D|B0002GY9M2|             Raxxess|This great 11-gau...|{null, null, null...|Digital Music|  7.99|        199541100788|Raxxess Rack Rail...|
|                 N/D|      N/D|B0002H0CW2|                    |                 N/D|{null, null, null...|Digital Music|  0.91|           595400100|  

In [10]:
df_meta.printSchema()

root
 |-- also_buy: string (nullable = false)
 |-- also_view: string (nullable = false)
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = false)
 |-- description: string (nullable = false)
 |-- details: struct (nullable = true)
 |    |-- \n    Item Weight: \n    : string (nullable = true)
 |    |-- \n    Product Dimensions: \n    : string (nullable = true)
 |    |-- ASIN:: string (nullable = true)
 |    |-- ASIN: : string (nullable = true)
 |    |-- Apparel: string (nullable = true)
 |    |-- Audio CD: string (nullable = true)
 |    |-- Audio Cassette: string (nullable = true)
 |    |-- Blu-ray Audio: string (nullable = true)
 |    |-- DVD: string (nullable = true)
 |    |-- DVD Audio: string (nullable = true)
 |    |-- Label:: string (nullable = true)
 |    |-- MP3 Music: string (nullable = true)
 |    |-- Note on Boxed Sets:: string (nullable = true)
 |    |-- Number of Discs:: string (nullable = true)
 |    |-- Original Release Date:: string (nullable = true)
 |    

In [11]:
df_meta2.printSchema()

root
 |-- also_buy: string (nullable = false)
 |-- also_view: string (nullable = false)
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = false)
 |-- description: string (nullable = false)
 |-- details: struct (nullable = true)
 |    |-- \n    Item Weight: \n    : string (nullable = true)
 |    |-- \n    Product Dimensions: \n    : string (nullable = true)
 |    |-- ASIN:: string (nullable = true)
 |    |-- ASIN: : string (nullable = true)
 |    |-- Batteries: string (nullable = true)
 |    |-- Discontinued by manufacturer:: string (nullable = true)
 |    |-- Domestic Shipping: : string (nullable = true)
 |    |-- International Shipping: : string (nullable = true)
 |    |-- Item model number:: string (nullable = true)
 |    |-- Shipping Advisory:: string (nullable = true)
 |    |-- Shipping Weight:: string (nullable = true)
 |    |-- UPC:: string (nullable = true)
 |-- main_cat: string (nullable = false)
 |-- price: float (nullable = true)
 |-- rank: string (nullable =

In [12]:
df_meta3.printSchema()

root
 |-- also_buy: string (nullable = false)
 |-- also_view: string (nullable = false)
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = false)
 |-- description: string (nullable = false)
 |-- details: struct (nullable = true)
 |    |-- \n    Item Weight: \n    : string (nullable = true)
 |    |-- \n    Product Dimensions: \n    : string (nullable = true)
 |    |-- ASIN:: string (nullable = true)
 |    |-- ASIN: : string (nullable = true)
 |    |-- Batteries: string (nullable = true)
 |    |-- Domestic Shipping: : string (nullable = true)
 |    |-- International Shipping: : string (nullable = true)
 |    |-- Item model number:: string (nullable = true)
 |    |-- Publisher:: string (nullable = true)
 |    |-- Shipping Advisory:: string (nullable = true)
 |    |-- Shipping Weight:: string (nullable = true)
 |    |-- UPC:: string (nullable = true)
 |-- main_cat: string (nullable = false)
 |-- price: float (nullable = true)
 |-- rank: string (nullable = true)
 |-- title: 

In [13]:
df_details1 = df_meta.select(
    "details.*"
)
df_details1.show()

+-------------------------+--------------------------------+----------+------+-------+-------------------+--------------+-------------+----+---------+-----------------+---------+-------------------+----------------+----------------------+------------+---------+-----------+----------------+----+-----+-----------+
|\n    Item Weight: \n    |\n    Product Dimensions: \n    |     ASIN:|ASIN: |Apparel|           Audio CD|Audio Cassette|Blu-ray Audio| DVD|DVD Audio|           Label:|MP3 Music|Note on Boxed Sets:|Number of Discs:|Original Release Date:|Please Note:|Run Time:|SPARS Code:|Shipping Weight:|UPC:|Vinyl|Vinyl Bound|
+-------------------------+--------------------------------+----------+------+-------+-------------------+--------------+-------------+----+---------+-----------------+---------+-------------------+----------------+----------------------+------------+---------+-----------+----------------+----+-----+-----------+
|                     null|                            nul

In [14]:
df_details2 = df_meta2.select(
    "details.*"
)
df_details2.show()

+-------------------------+--------------------------------+-----+----------+---------+-----------------------------+--------------------+------------------------+------------------+------------------+----------------+--------------------+
|\n    Item Weight: \n    |\n    Product Dimensions: \n    |ASIN:|    ASIN: |Batteries|Discontinued by manufacturer:| Domestic Shipping: |International Shipping: |Item model number:|Shipping Advisory:|Shipping Weight:|                UPC:|
+-------------------------+--------------------------------+-----+----------+---------+-----------------------------+--------------------+------------------------+------------------+------------------+----------------+--------------------+
|                     null|                            null| null|6546546450|     null|                         null|                null|                    null|              null|              null|            null|                null|
|                     null|            5

In [15]:
df_details3 = df_meta3.select(
    "details.*"
)
df_details3.show()

+-------------------------+--------------------------------+-----+------+---------+-------------------+------------------------+------------------+----------+------------------+----------------+----+
|\n    Item Weight: \n    |\n    Product Dimensions: \n    |ASIN:|ASIN: |Batteries|Domestic Shipping: |International Shipping: |Item model number:|Publisher:|Shipping Advisory:|Shipping Weight:|UPC:|
+-------------------------+--------------------------------+-----+------+---------+-------------------+------------------------+------------------+----------+------------------+----------------+----+
|                     null|                            null| null|  null|     null|               null|                    null|              null|      null|              null|            null|null|
|                     null|                            null| null|  null|     null|               null|                    null|              null|      null|              null|            null|null|
